<a href="https://colab.research.google.com/github/Tuxliri/RL_rocket/blob/test-PPO-module-registration/run_my_environment_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!cat ~/.ssh/id_ed25519.pub
PUB_KEY = "ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKRCMs6GPJ5kNi/fNVNf8Ki/QNwHS494mwuysriErNVs root@ac3a85e94cb7"

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKRCMs6GPJ5kNi/fNVNf8Ki/QNwHS494mwuysriErNVs root@ac3a85e94cb7


PRIVATE KEY:

In [27]:
!mkdir "my-environment"
!cd "my-environment"

GITHUB_PRIVATE_KEY = """b3BlbnNzaC1rZXktdjEAAAAABG5vbmUAAAAEbm9uZQAAAAAAAAABAAAAMwAAAAtzc2gtZW
QyNTUxOQAAACCkQjLOhjyeZDYv3zVTX/Cov0DcB0uPeJsLsrK4hKzVbAAAAJhh7cU0Ye3F
NAAAAAtzc2gtZWQyNTUxOQAAACCkQjLOhjyeZDYv3zVTX/Cov0DcB0uPeJsLsrK4hKzVbA
AAAEDN2iiEa8wQ9vr1YZa/Db3A25rqlIJmgS5/JHXuCZNpeqRCMs6GPJ5kNi/fNVNf8Ki/
QNwHS494mwuysriErNVsAAAAEXJvb3RAYWMzYTg1ZTk0Y2I3AQIDBA=="""

# Create the directory if it doesn't exist.
#! mkdir -p /root/.ssh
# Write the key
#with open("/root/.ssh/id_ed25519, "w") as f:
#  f.write(GITHUB_PRIVATE_KEY)
# Add github.com to our known hosts
! ssh-keyscan -t ed25519 github.com >> ~/.ssh/known_hosts
# Restrict the key permissions, or else SSH will complain.
! chmod go-rwx /root/.ssh/id_ed25519

# Note the `git@github.com` syntax, which will fetch over SSH instead of
# HTTP.
!git clone --branch test-PPO-module-registration git@github.com:Tuxliri/RL_rocket.git

#! git clone git@github.com:my_username/my_password.git

!cd ..

mkdir: cannot create directory ‘my-environment’: File exists
# github.com:22 SSH-2.0-babeld-7f91b4d6
Cloning into 'RL_rocket'...
remote: Enumerating objects: 636, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 636 (delta 133), reused 205 (delta 94), pack-reused 387
Receiving objects: 100% (636/636), 8.72 MiB | 5.64 MiB/s, done.
Resolving deltas: 100% (352/352), done.


In [34]:
!pip install stable-baselines3==1.5.0
%cd RL_rocket
import sys, os
sys.path.append(os.getcwd())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/RL_rocket


**IMPORT ALL THE NEEDED PACKAGES**

In [ ]:
from datetime import datetime
from genericpath import exists

import os
import sys
import gym
from my_environment.utils.callbacks import FigureRecorderCallback
import numpy as np
from stable_baselines3 import A2C, DQN, PPO, TD3
from tensorboard import program
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
import my_environment
from my_environment.wrappers.wrappers import DiscreteActions, DiscreteActions3DOF
from my_environment.envs import Rocket1D
from gym.wrappers import FlattenObservation, FilterObservation

# Execute the training

Setup tensorboard dashboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Choose the folder to store tensorboard logs
TENSORBOARD_LOGS_DIR = "RL_tests/my_environment/logs"

# Start tensorboard server
tb = program.TensorBoard()
tb.configure(argv=[None, '--logdir', TENSORBOARD_LOGS_DIR])
url = tb.launch()
print(f"Tensorboard listening on {url}")

**Environment definition**

Here we define the environment, setting up the initial conditions, the range over which the initial conditions are generated and the timestep of the simulation.

In [ ]:
env_id = "my_environment/Falcon3DOF-v0"
initial_conditions = [100, 500, np.pi/2, 0, -50, 0, 50e3]
initial_conditions_range = [0,0,0,0,0,0,0]
timestep = 0.1

env = gym.make(
    env_id,
    IC=initial_conditions,
    ICRange=initial_conditions_range,
    timestep=timestep,
    )

"""
env = DiscreteActions(Rocket1D(env,reward_type='test3dof'))
env = FlattenObservation(FilterObservation(env, ['observation']))
"""

# Define a new custom action space with only three actions:
# - no thrust
# - max thrust gimbaled right
# - max thrust gimbaled left
# - max thrust downwards
disc_to_cont = [
    [0, -1], [-1, +1],
    [0, +1], [+1, +1]
                ]
    
env = DiscreteActions3DOF(env, disc_to_cont)


env = Monitor(
    env,
    allow_early_resets=True,
    filename="logs_PPO"
    )


In [ ]:
##
model = PPO(
    'MlpPolicy',
    env,
    tensorboard_log=TENSORBOARD_LOGS_DIR,
    verbose=1
    )

# Train the agent
TRAINING_TIMESTEPS = 1e6

# Save the agent in the 'models' folder
date = datetime.now()
pathname = os.path.dirname(sys.argv[0])
savefolder = os.path.join(pathname, 'models')

if not exists(savefolder):
    os.mkdir(savefolder)

filename = "A2C_landing" + date.strftime("%Y-%m-%d_%H-%M")

callbacksList = [
    EvalCallback(
            model.get_env(),
            eval_freq = TRAINING_TIMESTEPS/10,
            n_eval_episodes=1,
            best_model_save_path=os.path.join(savefolder, filename),
            render=False,
            callback_after_eval=FigureRecorderCallback()
            )
        ]

model.learn(
    total_timesteps=TRAINING_TIMESTEPS,
    callback=callbacksList
)

rewards = evaluate_policy(model=model,env=model.get_env(),render=False,n_eval_episodes=10)

model.save()